In [2]:

vcf_file_address= "22_all.vcf"  #"22_allps_200.vcf"
vcf_file = open(vcf_file_address,'r');


first_het_variant_s1=True 
blockid_blocks_s1=[]
allele_blocks_s1=[]
varpos_blocks_s1=[]

first_het_variant_s2=True 
blockid_blocks_s2=[]
allele_blocks_s2=[]
varpos_blocks_s2=[]
var_blockids1_blocks_s2=[]   # for each var in s2, shows the s1 blockid


alleles_s5=[]
varposs_s5=[]

header_lines_list=[]
for line in vcf_file:
    line_strip=line.strip()
    if line_strip.startswith('#'):
        header_lines_list.append(line_strip)
        sample_names=line_strip.split('\t')[9:] # last line of header contains sample name
        #print(sample_names)
    else:

        line_parts=line_strip.split('\t')
        #gt_flags, s1, sss, ss, s2, s5 = line_parts[8:]
        gt_flags, s1, s2, s5 = line_parts[8:] # hifi illumina true

        varpos=int(line_parts[1]) # variant position
        gt_flags_split=gt_flags.split(":")
        
        if "PS" in gt_flags_split:
            idx_ps= gt_flags.split(":").index("PS")
            idx_gt= gt_flags.split(":").index("GT")
            
            
            # parental
            split_s5 = s5.split(":")
            blockid_s5 = split_s5[idx_ps]
            allele_s5 = split_s5[idx_gt]
            if allele_s5 !="./.":
                alleles_s5.append(int(allele_s5[0]))
                varposs_s5.append(varpos)

            
            split_s1=s1.split(":")
            blockid_s1 = split_s1[idx_ps]
            allele_s1=split_s1[idx_gt]
            
            if allele_s1=="0/1":allele_s1="./.";blockid_s1="."
                
            if not blockid_s1 == '.':
                if first_het_variant_s1:           # for the first het variant
                    first_het_variant_s1 = False
                    
                    allele_block_s1 = [int(allele_s1[0])]     # alleles of a block
                    varpos_block_s1 = [varpos]               # var pos of a block
                    blockid_blocks_s1.append(int(blockid_s1))   # add the blockid of the first var  to the list of blockids
                    
                else:                              # for the rest of het variants
                    if int(blockid_s1) in blockid_blocks_s1:
                        allele_block_s1.append(int(allele_s1[0]))
                        varpos_block_s1.append(varpos)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s1.append(allele_block_s1)
                        varpos_blocks_s1.append(varpos_block_s1)

                        # creat new phase block
                        allele_block_s1 = [int(allele_s1[0])]
                        varpos_block_s1 = [varpos]
                        blockid_blocks_s1.append(int(blockid_s1))

                        
            
            split_s2=s2.split(":")
            blockid_s2 = split_s2[idx_ps]
            allele_s2=split_s2[idx_gt]
            
            if allele_s2=="0/1":allele_s2="./.";blockid_s2="."
                
            if not blockid_s2 == '.':
                if first_het_variant_s2:           # for the first het variant
                    first_het_variant_s2 = False
                    
                    allele_block_s2 = [int(allele_s2[0])]     # alleles of a block
                    varpos_block_s2 = [varpos]               # var pos of a block
                    blockid_blocks_s2.append(int(blockid_s2))   # add the blockid of the first var  to the list of blockids
                    var_blockids1_block_s2=[blockid_s1]
                    
                else:                              # for the rest of het variants
                    if int(blockid_s2) in blockid_blocks_s2:
                        allele_block_s2.append(int(allele_s2[0]))
                        varpos_block_s2.append(varpos)
                        var_blockids1_block_s2.append(blockid_s1)
                        
                    else: 
                        # add previous block to the list of all blocks
                        allele_blocks_s2.append(allele_block_s2)
                        varpos_blocks_s2.append(varpos_block_s2)
                        var_blockids1_blocks_s2.append(var_blockids1_block_s2)

                        # creat new phase block
                        allele_block_s2 = [int(allele_s2[0])]
                        varpos_block_s2 = [varpos]
                        blockid_blocks_s2.append(int(blockid_s2))
                        var_blockids1_block_s2=[blockid_s1]
                       
                        
                        
# add last block to the list of all blocks
allele_blocks_s1.append(allele_block_s1)
varpos_blocks_s1.append(varpos_block_s1)

allele_blocks_s2.append(allele_block_s2)
varpos_blocks_s2.append(varpos_block_s2)
var_blockids1_blocks_s2.append(var_blockids1_block_s2)
                        
    
    
number_phaseblocks_s2= len(allele_blocks_s2)
print(number_phaseblocks_s2)

number_phaseblocks_s1= len(allele_blocks_s1)
print(number_phaseblocks_s1)
 
    

        

6771
252


In [3]:
#%pwd

for allele_block_s1 in allele_blocks_s1:
    print(len(allele_block_s1))

103
163
668
789
723
346
14
397
29
3
109
8
1241
47
115
103
263
2340
262
212
521
21
10
8
140
112
85
419
107
81
75
27
27
23
520
491
2
119
7
32
96
41
70
57
25
19
8
5
2
151
193
302
20
723
16
5
3
3
8
181
24
3
375
182
403
14
233
18
7
3
5
443
3
6
2
207
198
211
125
449
2
927
111
13
19
4
11
2
2
2
5
3
10
2
39
2
511
8
5
116
834
4
81
85
3
3
1
2
253
580
3
3
11
3
14
6
362
653
16
25
50
147
16
60
2
90
101
204
9
94
18
9
14
13
3
2
11
2
23
147
6
6
5
10
2
15
448
5
7
1
18
7
3
631
4
79
307
214
164
575
200
15
3
3
44
106
40
2
7
22
6
199
27
450
3
14
81
212
2
6
4
2
73
276
72
17
2
8
12
20
12
12
297
9
6
8
6
4
12
8
2
9
6
6
20
589
10
56
3
1967
170
156
77
5
11
1235
85
136
30
319
10
280
57
762
133
8
39
792
1
33
11
240
5
24
228
2
25
3
75
201
290
517
1186
13
311
20
4
2
39
509
25
661


In [4]:
intersected_blocks_idins1_s2=[] # for each block in s2, the corresponding s1 block id

for var_blockids1_blocks_s2_i in var_blockids1_blocks_s2:
    var_blockids1_blocks_s2_i_set=set(var_blockids1_blocks_s2_i)
    
    if '.' in var_blockids1_blocks_s2_i_set:
        var_blockids1_blocks_s2_i_set.discard('.')
        
    if len(var_blockids1_blocks_s2_i_set)==1:
        intersected_idins1_s2=int(list(var_blockids1_blocks_s2_i_set)[0])
    if len(var_blockids1_blocks_s2_i_set)==0:
        intersected_idins1_s2=0
    if len(var_blockids1_blocks_s2_i_set)>1:
        # find the most frequent blockid in the phaseblock
        intersected_idins1_s2=int(max(var_blockids1_blocks_s2_i_set, key=var_blockids1_blocks_s2_i.count) ) 
        
    intersected_blocks_idins1_s2.append(intersected_idins1_s2)


In [5]:
#print(blockid_blocks_s1)
#for i in range(len(allele_blocks_s1)):
#    print(len(allele_blocks_s1[i]))
    
#print(blockid_blocks_s2)
#for i in range(len(allele_blocks_s2)):
#    print(len(allele_blocks_s2[i]))

In [6]:
number_phaseblocks_s1= len(allele_blocks_s1)

allele_blocks_s1_new=list(allele_blocks_s1)
for block_i in range(number_phaseblocks_s1): # 
   
    allele_block_s1=allele_blocks_s1[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    similiar_count=0
    diss_count=0
    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
            if allele_s1 == allele_s5:
                similiar_count+=1
            else: 
                diss_count+=1
                
    print(similiar_count, diss_count)          
    if similiar_count < diss_count:
        allele_block_s1_new = [1-i for i in allele_block_s1]
        allele_blocks_s1_new[block_i]=allele_block_s1_new
        


12 7
16 23
39 35
35 242
2 256
84 67
3 0
1 56
1 7
0 1
0 49
1 4
0 874
4 0
1 8
0 16
23 23
259 1082
3 5
12 15
46 245
2 6
4 0
0 1
0 44
20 7
8 11
17 75
1 10
10 2
0 5
4 0
1 11
4 1
227 0
33 60
0 0
44 3
3 0
5 0
0 8
0 13
52 4
0 16
0 6
2 0
1 0
4 0
1 0
135 0
1 103
1 75
0 17
200 6
7 0
0 1
2 0
0 0
3 0
12 9
0 13
1 0
8 99
82 0
333 4
6 2
72 123
0 0
0 7
0 1
1 0
0 305
2 0
0 3
0 1
0 50
59 0
0 82
0 44
0 250
0 0
182 321
0 83
7 2
19 0
0 4
0 9
0 2
2 0
2 0
5 0
3 0
0 6
0 0
1 6
0 0
1 329
0 7
3 0
8 79
431 0
0 2
19 0
22 2
0 1
1 0
0 0
0 2
119 0
343 2
0 1
0 0
0 5
0 0
2 0
2 0
5 215
273 0
9 0
0 18
0 10
82 1
0 12
29 0
0 2
77 0
88 0
0 104
0 4
75 0
0 13
1 2
0 5
2 1
2 0
2 0
0 0
0 0
0 4
62 0
0 1
0 4
0 1
1 3
1 0
0 2
1 334
1 0
5 0
0 0
0 14
7 0
0 1
295 50
0 0
6 39
5 148
142 0
71 2
116 49
0 73
0 2
1 0
0 0
13 0
46 0
0 26
1 0
4 0
11 0
5 0
97 0
0 24
5 282
0 1
0 4
65 0
6 78
0 1
0 2
3 0
1 0
32 0
0 256
0 41
1 1
0 1
1 0
2 0
1 7
0 4
0 1
13 0
0 2
1 0
3 0
0 2
1 0
3 1
1 1
1 0
0 3
0 1
3 0
1 4
242 55
0 3
0 26
0 0
359 961
87 0
71 13
38 0
0 

In [7]:

allele_blocks_s2_new=list(allele_blocks_s2)
for block_i in range(number_phaseblocks_s2): # 
#for block_i, allele_block_s2 in enumerate(allele_blocks_s2):
   
    allele_block_s2=allele_blocks_s2[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1_new[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)
    similiar_count=0
    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: similiar_count+=1
    #print(similiar_count/len(intersect_varpos))
    if similiar_count < len(intersect_varpos)/2:
        #print("need", similiar_count)
        allele_block_s2_new= [1-i for i in allele_block_s2]
        allele_blocks_s2_new[block_i]=allele_block_s2_new
        


In [12]:
len(blockid_blocks_s1)

252

In [13]:
matrix_address="matrix_up2_"+vcf_file_address+".txt"

matrix_file=open(matrix_address,'w');
matrix_file.write("\t\t".join(["Pos","HiFi","Illm","parental"])+"\n") # "Pos","ONT","CLR","HiFi","Illm","parental"


for block_i_s1 in range(len(blockid_blocks_s1)):
    blockid_block_s1=blockid_blocks_s1[block_i_s1]
    
    allele_block_s1=allele_blocks_s1_new[block_i_s1]
    varpos_block_s1=varpos_blocks_s1[block_i_s1]
    for var_i_s1 in range(len(varpos_block_s1)):
        varpos=varpos_block_s1[var_i_s1]
        allele_s1=allele_block_s1[var_i_s1]
        
        allele_s2='.'
        blockid_s2='.'
        for block_i_s2 in range(len(blockid_blocks_s2)):
            blockid_block_s2 = blockid_blocks_s2[block_i_s2]
            allele_block_s2 = allele_blocks_s2_new[block_i_s2]
            varpos_block_s2 = varpos_blocks_s2[block_i_s2]
            
            if varpos in varpos_block_s2:
                allele_s2 = allele_block_s2[varpos_block_s2.index(varpos)]
                blockid_s2=blockid_block_s2
                break # when var is found, no need to continue for loop
            
        if varpos in varposs_s5:
            allele_s5 = alleles_s5[varposs_s5.index(varpos)]
        else: allele_s5= '.'

                
        matrix_row_list=[str(varpos),str(allele_s1)+":"+str(blockid_block_s1),
                         str(allele_s2)+":"+str(blockid_s2),str(allele_s5)] 
        matrix_file.write("\t".join(matrix_row_list)+"\n")
    
matrix_file.close()


In [14]:
len(blockid_blocks_s1)

252

[16051453,
 16056353,
 16062995,
 16065078,
 16065536,
 16067208,
 16068295,
 16069374,
 16073196,
 16077252]

In [18]:

diff_list=[]

for block_i in range(len(allele_blocks_s1_new)): # 
   
    allele_block_s1=allele_blocks_s1_new[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s1!=allele_s5:
                
                for block_i in range(number_phaseblocks_s2): # 
   
                    allele_block_s2=allele_blocks_s2_new[block_i]
                    varpos_block_s2=varpos_blocks_s2[block_i]
                    if varpos in varpos_block_s2:
                        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
                        diff_list.append([varpos,allele_s1,allele_s2,allele_s5])
                        print([varpos,allele_s1,allele_s2,allele_s5],"\n")
                        break

                
                
                
                
                



[16069374, 0, 0, 1] 

[16080579, 0, 0, 1] 

[16086302, 0, 0, 1] 

[16087217, 0, 0, 1] 

[16101684, 1, 1, 0] 

[16104385, 1, 1, 0] 

[16105717, 1, 0, 0] 

[16113547, 0, 0, 1] 

[16117006, 0, 1, 1] 

[16120074, 1, 0, 0] 

[16123812, 1, 0, 0] 

[16137152, 1, 0, 0] 

[16142002, 1, 1, 0] 

[16143370, 1, 1, 0] 

[16143995, 1, 1, 0] 

[16144768, 1, 1, 0] 

[16144952, 1, 1, 0] 

[16149963, 1, 0, 0] 

[16159540, 0, 0, 1] 

[16164946, 1, 1, 0] 

[16183780, 1, 1, 0] 

[16183790, 1, 1, 0] 

[16194681, 1, 0, 0] 

[16199059, 1, 0, 0] 

[16201114, 1, 0, 0] 

[16201332, 1, 0, 0] 

[16201360, 1, 0, 0] 

[16203241, 1, 1, 0] 

[16203542, 0, 1, 1] 

[16204790, 1, 0, 0] 

[16204824, 1, 0, 0] 

[16207034, 1, 0, 0] 

[16212603, 1, 0, 0] 

[16213011, 1, 0, 0] 

[16214176, 1, 1, 0] 

[16216383, 1, 1, 0] 

[16217414, 1, 1, 0] 

[16217477, 1, 1, 0] 

[16219445, 1, 0, 0] 

[16223201, 1, 0, 0] 

[16228164, 1, 1, 0] 

[16228270, 1, 1, 0] 

[16230082, 1, 1, 0] 

[16230525, 1, 1, 0] 

[16231828, 1, 0, 0] 

[16240092,

[20343349, 1, 0, 0] 

[20627525, 1, 0, 0] 

[20639324, 0, 0, 1] 

[20639332, 0, 0, 1] 

[20639571, 0, 0, 1] 

[20640223, 0, 0, 1] 

[20647060, 0, 0, 1] 

[20647614, 1, 1, 0] 

[20986299, 1, 1, 0] 

[20988045, 1, 1, 0] 

[20988390, 1, 1, 0] 

[20988749, 1, 1, 0] 

[20989409, 1, 1, 0] 

[20989749, 1, 1, 0] 

[20989898, 1, 1, 0] 

[20990057, 1, 1, 0] 

[20991062, 1, 1, 0] 

[20991129, 1, 1, 0] 

[20991235, 1, 1, 0] 

[20991771, 1, 1, 0] 

[20993467, 0, 0, 1] 

[20993519, 0, 0, 1] 

[20993797, 0, 0, 1] 

[20994344, 1, 1, 0] 

[20996595, 1, 1, 0] 

[20997232, 1, 1, 0] 

[20998885, 1, 1, 0] 

[20999190, 1, 1, 0] 

[20999208, 1, 1, 0] 

[21000580, 1, 1, 0] 

[21001008, 0, 0, 1] 

[21001074, 1, 1, 0] 

[21001422, 1, 1, 0] 

[21002437, 1, 1, 0] 

[21002497, 1, 1, 0] 

[21002822, 0, 0, 1] 

[21002886, 1, 1, 0] 

[21003110, 0, 0, 1] 

[21003446, 0, 0, 1] 

[21003490, 1, 1, 0] 

[21003610, 0, 0, 1] 

[21004056, 0, 0, 1] 

[21004441, 0, 0, 1] 

[21012568, 1, 0, 0] 

[21017767, 0, 0, 1] 

[21017919,

[37234253, 1, 1, 0] 

[37355706, 0, 0, 1] 

[37842927, 0, 0, 1] 

[37842933, 0, 0, 1] 

[37843239, 0, 0, 1] 

[37843260, 0, 0, 1] 

[37845116, 0, 0, 1] 

[37845452, 0, 0, 1] 

[37846010, 0, 0, 1] 

[37857126, 0, 0, 1] 

[37857279, 0, 0, 1] 

[37887241, 1, 1, 0] 

[37898411, 0, 0, 1] 

[37898896, 0, 0, 1] 

[37899101, 0, 0, 1] 

[37900771, 0, 0, 1] 

[37900983, 0, 0, 1] 

[37904695, 0, 0, 1] 

[37905173, 0, 0, 1] 

[38053808, 0, 0, 1] 

[38079095, 0, 0, 1] 

[38179890, 0, 0, 1] 

[38187778, 0, 0, 1] 

[38197028, 1, 1, 0] 

[38198314, 0, 1, 1] 

[38225830, 0, 0, 1] 

[38242187, 1, 1, 0] 

[38250886, 0, 0, 1] 

[38283643, 1, 1, 0] 

[38292842, 1, 1, 0] 

[38306452, 0, 0, 1] 

[38353946, 0, 0, 1] 

[39387033, 1, 1, 0] 

[39387526, 1, 1, 0] 

[39387531, 1, 1, 0] 

[39903457, 1, 1, 0] 

[39930341, 1, 1, 0] 

[39933414, 1, 1, 0] 

[39947225, 0, 1, 1] 

[40912244, 1, 0, 0] 

[42023810, 0, 1, 1] 

[42531392, 1, 0, 0] 

[42531399, 1, 0, 0] 

[42716632, 1, 1, 0] 

[42716840, 1, 1, 0] 

[42720463,

[45988612, 0, 0, 1] 

[45989072, 0, 0, 1] 

[45989440, 0, 0, 1] 

[45989594, 0, 0, 1] 

[45990722, 0, 0, 1] 

[45990786, 0, 0, 1] 

[45990805, 0, 0, 1] 

[45990901, 0, 0, 1] 

[45991405, 0, 0, 1] 

[45992424, 0, 0, 1] 

[45992485, 0, 0, 1] 

[45992976, 0, 0, 1] 

[45993072, 0, 0, 1] 

[45993422, 0, 0, 1] 

[45993549, 0, 0, 1] 

[45993858, 0, 0, 1] 

[45994167, 0, 0, 1] 

[45994189, 0, 0, 1] 

[45994388, 0, 0, 1] 

[45994503, 0, 0, 1] 

[45994609, 0, 0, 1] 

[45994667, 0, 0, 1] 

[45995356, 0, 0, 1] 

[45995792, 0, 0, 1] 

[45996110, 0, 0, 1] 

[45998404, 0, 0, 1] 

[45998599, 0, 0, 1] 

[45998735, 0, 0, 1] 

[45998785, 0, 0, 1] 

[45998841, 0, 0, 1] 

[45998867, 0, 0, 1] 

[45998943, 0, 0, 1] 

[45999071, 0, 0, 1] 

[45999123, 0, 0, 1] 

[46002104, 0, 0, 1] 

[46002768, 0, 0, 1] 

[46003232, 0, 0, 1] 

[46004780, 1, 1, 0] 

[46004878, 0, 0, 1] 

[46897904, 0, 1, 1] 

[46902787, 0, 0, 1] 

[46904273, 0, 1, 1] 

[46905858, 1, 1, 0] 

[46906000, 1, 1, 0] 

[46907779, 1, 1, 0] 

[46909355,

In [58]:

diff_list_s2=[]

for block_i in range(len(allele_blocks_s2_new)): # 
   
    allele_block_s2=allele_blocks_s2_new[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]


    for var_i, varpos in enumerate(varpos_block_s2):
        allele_s2= allele_block_s2[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s2!=allele_s5:
                
                for block_i in range(number_phaseblocks_s1): # 
   
                    allele_block_s1=allele_blocks_s1_new[block_i]
                    varpos_block_s1=varpos_blocks_s1[block_i]
                    if varpos in varpos_block_s1:
                        allele_s1= allele_block_s1[varpos_block_s2.index(varpos)]
                        diff_list_s2.append([varpos,allele_s1,allele_s2,allele_s5])
                        print([varpos,allele_s1,allele_s2,allele_s5],"\n")
                        break

            

[16069374, 0, 0, 1] 

[16073196, 0, 0, 1] 

[16080579, 0, 0, 1] 

[16086302, 0, 0, 1] 

[16087217, 0, 0, 1] 

[16101684, 1, 1, 0] 

[16104385, 0, 1, 0] 

[16105128, 1, 0, 1] 

[16113547, 1, 0, 1] 

[16127345, 1, 0, 1] 

[16138401, 1, 1, 0] 

[16142002, 1, 1, 0] 

[16142229, 1, 1, 0] 

[16142899, 1, 1, 0] 

[16143370, 0, 1, 0] 

[16143995, 1, 1, 0] 

[16144768, 1, 1, 0] 

[16144952, 1, 1, 0] 

[16147765, 0, 1, 0] 

[16150751, 1, 0, 1] 

[16159540, 1, 0, 1] 

[16160468, 1, 0, 1] 

[16164946, 1, 1, 0] 

[16173634, 1, 1, 0] 

[16183780, 1, 1, 0] 

[16183790, 1, 1, 0] 

[16203241, 1, 1, 0] 

[16212160, 1, 0, 1] 

[16214176, 1, 1, 0] 

[16216190, 0, 0, 1] 

[16216383, 0, 1, 0] 

[16217414, 0, 1, 0] 

[16217477, 1, 1, 0] 

[16219080, 0, 0, 1] 

[16219632, 1, 0, 1] 

[16223527, 1, 0, 1] 

[16228164, 1, 1, 0] 

[16228270, 1, 1, 0] 

[16230082, 1, 1, 0] 

[16230525, 1, 1, 0] 

[16240092, 1, 1, 0] 

[16246137, 1, 0, 1] 

[16258516, 0, 1, 0] 

[16267436, 0, 1, 0] 

[16278102, 1, 1, 0] 

[16279410,

[20202959, 1, 0, 1] 

[20204036, 1, 0, 1] 

[20204159, 1, 0, 1] 

[20209030, 1, 0, 1] 

[20209425, 1, 1, 0] 

[20209487, 1, 0, 1] 

[20212954, 1, 0, 1] 

[20217036, 1, 1, 0] 

[20217203, 1, 0, 1] 

[20217680, 1, 0, 1] 

[20218997, 1, 1, 0] 

[20219480, 1, 0, 1] 

[20221844, 1, 1, 0] 

[20221963, 1, 1, 0] 

[20233231, 1, 0, 1] 

[20233456, 1, 1, 0] 

[20234525, 1, 0, 1] 

[20245174, 1, 0, 1] 

[20248391, 1, 1, 0] 

[20248713, 1, 1, 0] 

[20249421, 1, 1, 0] 

[20249551, 1, 1, 0] 

[20249807, 1, 1, 0] 

[20263548, 1, 0, 1] 

[20264101, 1, 1, 0] 

[20264301, 1, 1, 0] 

[20266415, 1, 0, 1] 

[20266472, 1, 0, 1] 

[20267689, 1, 1, 0] 

[20268134, 1, 0, 1] 

[20268212, 1, 0, 1] 

[20268787, 0, 0, 1] 

[20270367, 1, 0, 1] 

[20272895, 1, 0, 1] 

[20272900, 1, 0, 1] 

[20273404, 1, 1, 0] 

[20273460, 0, 0, 1] 

[20273477, 0, 0, 1] 

[20275368, 1, 0, 1] 

[20275382, 1, 0, 1] 

[20275636, 1, 0, 1] 

[20275685, 1, 0, 1] 

[20275761, 1, 0, 1] 

[20275770, 0, 0, 1] 

[20280439, 1, 0, 1] 

[20280782,

[29753322, 1, 1, 0] 

[29775618, 1, 0, 1] 

[29775797, 1, 1, 0] 

[29776323, 1, 1, 0] 

[29776857, 1, 1, 0] 

[29806222, 1, 1, 0] 

[30753628, 1, 0, 1] 

[30970972, 0, 0, 1] 

[31323900, 0, 1, 0] 

[31671882, 0, 0, 1] 

[31672119, 1, 0, 1] 

[32456145, 0, 1, 0] 

[32576937, 0, 0, 1] 

[32927360, 1, 1, 0] 

[32928105, 0, 1, 0] 

[32928252, 0, 1, 0] 

[32928414, 0, 1, 0] 

[32928451, 0, 1, 0] 

[32928521, 0, 1, 0] 

[34792117, 1, 1, 0] 

[36087192, 1, 0, 1] 

[36087286, 0, 0, 1] 

[36087564, 1, 0, 1] 

[36087633, 1, 0, 1] 

[36088146, 1, 0, 1] 

[36088192, 1, 0, 1] 

[36088371, 1, 0, 1] 

[36088480, 1, 0, 1] 

[36088507, 1, 0, 1] 

[36088776, 1, 0, 1] 

[36088805, 1, 0, 1] 

[36088820, 1, 0, 1] 

[36089136, 1, 0, 1] 

[36089560, 1, 0, 1] 

[36089649, 1, 0, 1] 

[36089825, 1, 0, 1] 

[36090044, 1, 0, 1] 

[36090060, 1, 0, 1] 

[36713531, 1, 0, 1] 

[36811503, 0, 1, 0] 

[36943544, 1, 0, 1] 

[36944165, 0, 0, 1] 

[36944230, 1, 0, 1] 

[36944356, 1, 0, 1] 

[36945022, 0, 0, 1] 

[36945289,

[44290697, 0, 0, 1] 

[44290856, 0, 0, 1] 

[44291020, 0, 0, 1] 

[44291299, 0, 0, 1] 

[44293654, 1, 0, 1] 

[44295103, 1, 1, 0] 

[44295495, 0, 1, 0] 

[44300299, 1, 1, 0] 

[44300482, 1, 1, 0] 

[44300640, 1, 1, 0] 

[44300847, 1, 0, 1] 

[44301176, 0, 0, 1] 

[44301263, 0, 0, 1] 

[44301272, 0, 0, 1] 

[44302337, 1, 0, 1] 

[44302347, 0, 0, 1] 

[44302788, 1, 0, 1] 

[44511309, 0, 1, 0] 

[44603527, 1, 1, 0] 

[44694612, 1, 0, 1] 

[44694676, 1, 0, 1] 

[44695050, 1, 0, 1] 

[44695088, 1, 0, 1] 

[44696052, 1, 0, 1] 

[44696374, 1, 0, 1] 

[44700028, 1, 0, 1] 

[44700267, 1, 0, 1] 

[44701046, 1, 1, 0] 

[44702874, 0, 1, 0] 

[45047360, 0, 0, 1] 

[45047411, 0, 1, 0] 

[45047535, 0, 0, 1] 

[45047827, 1, 0, 1] 

[45048253, 1, 1, 0] 

[45049498, 0, 1, 0] 

[45049603, 0, 0, 1] 

[45083750, 0, 1, 0] 

[45280090, 1, 0, 1] 

[45280379, 1, 0, 1] 

[45379909, 0, 0, 1] 

[45557803, 1, 1, 0] 

[45624439, 0, 1, 0] 

[45624707, 0, 0, 1] 

[45624733, 0, 0, 1] 

[45746097, 0, 1, 0] 

[45935056,

In [65]:
hifiok_illno=[]
hifino_illno=[]
hifino_illok=[]

for [varpos, hifi,illu,parnt] in diff_list_s2:
    if illu!=parnt and hifi==parnt:
        hifiok_illno.append(varpos)
    if illu!=parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illno.append(varpos)       
    if illu==parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illok.append(varpos) 
    
print(len(diff_list_s2),len(hifiok_illno),len(hifino_illok),len(hifino_illno))

1655 786 0 869


In [66]:
hifiok_illno=[]
hifino_illno=[]
hifino_illok=[]

for [varpos, hifi,illu,parnt] in diff_list:
    
    if illu!=parnt and hifi==parnt:
        hifiok_illno.append(varpos)
    if illu!=parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illno.append(varpos)       
    if illu==parnt and hifi!=parnt:
        #print([varpos, hifi,illu,parnt] )
        hifino_illok.append(varpos) 
    
print(len(diff_list_s2),len(hifiok_illno),len(hifino_illok),len(hifino_illno))

1655 0 94 1335


In [33]:
allele_s5==allele_s2
[allele_s1,allele_s5,allele_s2]

True

In [76]:
similiar_count=0
dissimiliar_count=0
dissimiliar_count_s1ok=0
dissimiliar_count_s2ok=0
dissimiliar_count_exist5=0
similiar_count_s1s2ok=0
similiar_count_s1s2no=0
similiar_count_exist5=0

for block_i in range(number_phaseblocks_s2): # 
   
    allele_block_s2=allele_blocks_s2_new[block_i]
    varpos_block_s2=varpos_blocks_s2[block_i]

    intersected_idins1_s2=intersected_blocks_idins1_s2[block_i]
    if intersected_idins1_s2 !=0:
        block_indx_s1= blockid_blocks_s1.index(intersected_idins1_s2)
    varpos_block_s1=varpos_blocks_s1[block_indx_s1]
    allele_block_s1=allele_blocks_s1_new[block_indx_s1]
    
    intersect_varpos=set(varpos_block_s1) & set(varpos_block_s2)

    for varpos in intersect_varpos:
        allele_s1= allele_block_s1[varpos_block_s1.index(varpos)]
        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
        if allele_s1==allele_s2: 
            similiar_count+=1
            if varpos in varposs_s5:
                allele_s5= alleles_s5[varposs_s5.index(varpos)]
                similiar_count_exist5+=1
                if allele_s1==allele_s5 and allele_s2==allele_s5:
                    similiar_count_s1s2ok+=1
                if allele_s1!=allele_s5 and allele_s2!=allele_s5:
                    similiar_count_s1s2no+=1
        else:
            #print(varpos)
            dissimiliar_count+=1
            if varpos in varposs_s5:
                allele_s5= alleles_s5[varposs_s5.index(varpos)]
                dissimiliar_count_exist5+=1
                if allele_s1==allele_s5 and allele_s2!=allele_s5:
                    dissimiliar_count_s1ok+=1
                if allele_s1!=allele_s5 and allele_s2==allele_s5:
                    dissimiliar_count_s2ok+=1
                    

            
#     if similiar_count>1 and dissimiliar_count>1:
#         print(similiar_count)

print(similiar_count,dissimiliar_count,dissimiliar_count_exist5,dissimiliar_count_s1ok,dissimiliar_count_s2ok)
print(similiar_count_exist5,similiar_count_s1s2ok,similiar_count_s1s2no)

28012 2000 414 320 94
14005 12670 1335


In [13]:
len(allele_block_s2)

3

In [ ]:

diff_list=[]

for block_i in range(len(allele_blocks_s1_new)): # 
   
    allele_block_s1=allele_blocks_s1_new[block_i]
    varpos_block_s1=varpos_blocks_s1[block_i]


    for var_i, varpos in enumerate(varpos_block_s1):
        allele_s1= allele_block_s1[var_i]
        
        if varpos in varposs_s5:
            allele_s5= alleles_s5[varposs_s5.index(varpos)]
        
            if allele_s1!=allele_s5:
                
                for block_i in range(number_phaseblocks_s2): # 
   
                    allele_block_s2=allele_blocks_s2_new[block_i]
                    varpos_block_s2=varpos_blocks_s2[block_i]
                    if varpos in varpos_block_s2:
                        allele_s2= allele_block_s2[varpos_block_s2.index(varpos)]
                        diff_list.append([varpos,allele_s1,allele_s2,allele_s5])
                        print([varpos,allele_s1,allele_s2,allele_s5],"\n")
                        break

                
                
                